# Lab 6: replicate spatial accessibility

In this lab, you will choose **your own study area** to measure spatial accessibility to hospitals. The focus of this lab is to let you conduct an accessibility study **from scratch**, starting from data acquisition to accessibility measurement. <br>
The size of analysis is recommended to a county, given the analysis time. For example, you can investigate your home county. The data source for this lab is as below. 

* Supply: <a href=https://hifld-geoplatform.opendata.arcgis.com/> Homeland Infrastructure Foundation-Level Data (HIFLD) </a> - <a href=https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::hospitals-1/>Hospitals</a>.
* Demand: US Census Bureau - <a href="https://data.census.gov/cedsci/table?q=population">Decennial Census - Race </a>
* Geographical Area of the reference: <a href=https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html>Topologically Integrated Geographic Encoding and Referencing (TIGER) data </a>

The example below aims to measure hospital accessibility in Harris County, Texas. You can follow the instruction or come up with your own study area. 


In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import time
from tqdm import tqdm, trange
from shapely.geometry import Point, MultiPoint
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Data Preprocessing
### 1. (1.5 point) Demand-related data
#### 1.1. (0.5 point) Obtain geographical area of the reference
**1.1.1.** Click <a href=https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html>this link</a> to obtain census tract data from TIGER file.  <br>
**1.1.2.** Load the shapefile as a GeoDataFrame with the name of `tract`. <br>
**1.1.3.** Remove unnecessary information from the file and maintain only two columns (`GEOID` and `geometry`). <br>
**1.1.4.** Visit <a href="https://www.nrcs.usda.gov/wps/portal/nrcs/detail/tx/technical/dma/rwa/?cid=nrcs143_013697"> this website </a> and find the FIPS code of the county that you focus on. <br>
**1.1.5.** Slice the dataframe so that it only stores values associated with the current study area. 

**From this**
![](./data/fig1_1.jpg)

**to this**
![](./data/fig1_2.jpg)


In [ ]:
# Your code here



In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert type(tract) == gpd.GeoDataFrame
assert ('GEOID' in tract.columns) & ('geometry' in tract.columns)
assert ('STATEFP' not in tract.columns) & ('NAME' not in tract.columns)

print('Success!')

#### 1.2. (0.5 point) Obtain population data

**1.2.1.** Visit <a href="https://data.census.gov/cedsci/table?q=population">this website</a> and find the appropriate data according to your study area. <br>
**1.2.2.** Open the data as DataFrame with the variable name `pop`, and format the data as shown below. 

**From this**
![](./data/fig2_1.jpg)

**to this**
![](./data/fig2_2.jpg)

**1.2.3.** This will require the following progress. 
* Load the first row of the CSV as the header, but remove the first row of the data. 
* Select only the necessary information (`GEO_ID` and `P1_001N` columns).
* Rename the `P1_001N` column to `pop` with <a href=https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html>`df.rename()`</a>.
* From the `GEO_ID` column, remove the unnecessary digits (i.e., 1400000US) with <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html">`df.apply(lambda x:)`</a>. 
* Change the datatype of `pop` column from `str` to `int` (Currently, the population data is stored as a string) with <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html">`Series.astype()`</a>. 


In [ ]:
# Your code here



In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert type(pop) == pd.DataFrame
assert len(pop.at[1, 'GEO_ID']) == 11
assert ('GEO_ID' in pop.columns) & ('pop' in pop.columns)
assert pop['pop'].dtype == int

print('Success!')

#### 1.3. (0.5 point) Create `demand` GeoDataFrame based on `tract` GeoDataFrame and `pop` DataFrame. 
**1.3.1.** Investigate the two dataframe and see which column has the same data. <br>
**1.3.2.** Merge `tract` GeoDataFrame and `pop` DataFrame based on the columns you found with <a href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html>gpd.merge()</a> and name the resulted GeoDataFrame as `demand`. <br>
**1.3.3.** Change the coordinate system of the GeoDataFrame to "NAD83 / Conus Albers (epsg:5070)".<br>
**1.3.4.** Drop unnecessary `GEO_ID` column. The result should look like the below. <br>

![](./data/fig2_3.jpg)

In [ ]:
# Your code here



In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert type(demand) == gpd.GeoDataFrame
assert ('GEOID' in demand.columns) & ('pop' in demand.columns)
assert demand['pop'].dtype == int
assert demand.crs.name == 'NAD83 / Conus Albers'

print('Success!')

### 2. (1 point) Supply-related data

**2.1.** Visit <a href="https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::hospitals-1/explore">HIFLD website</a> and download the hospital data as shapefile to your local directory. <br>
**2.2.** Load the downloaded hospitals as GeoDataFrame with the name of `supply`. <br>
**2.3.** Change the coordinate system of `supply` to "NAD83 / Conus Albers (epsg:5070)", in order to match its dataframe with `demand` GeoDataFrame. <br>
**2.4.** Select only the records in `supply` GeoDataFrame within the geometry of `demand`. <br>
**2.5.** The downloaded data has duplicates so remove the duplicates based on the `ID` column by using <a href=https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html>`df.drop_duplicates()`</a>. <br>
**2.6.** Make `supply` GeoDataFrame only has the columns (`ID`, `NAME`, `CITY`, `ZIP`, `BEDS`, `geometry`). <br>
**2.7.** The data uses `-999` in `BEDS` column for the missing values. Remove the entire row if it has the missing value of `BEDS`.<br>
**2.8.** Reset the index of the current GeoDataFrame with <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html">`gdf.reset_index()`</a> function. <br>

The result should look like the below. 
![](./data/fig2_4.jpg)

In [ ]:
# Your code here



In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert type(supply) == gpd.GeoDataFrame
assert supply.crs.name == 'NAD83 / Conus Albers'
assert supply.shape == (115, 6)
assert supply.loc[supply['BEDS'] == 999].shape[0] == 0

print('Success!')

### 3. (0.5 point) Mobility-related data

Examine the following code and change the location according to your study area. 

```python
def remove_uncenessary_nodes(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg == 0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg == 0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component) < 10:
            for node in component:
                _nodes_removed += 1
                network.remove_node(node)

    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed / float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))

    return network


# Obtain OSM Network in Harris County, Texas
G = ox.graph_from_place('Harris County, TX, USA', network_type='drive', simplify=True)

# Simplify Graph: Remove edges
for u, v, data in G.copy().edges(data=True):
    if data['highway'] not in ['motorway', 'motorway_link', 
                               'trunk', 'trunk_link',
                               'primary', 'primary_link', 
# This line is commented to make the future process faster. 
# If your study area is small and doesn't have enough road network, please uncomment the following line. 
                               'secondary', 'secondary_link'  
                              ]:
        G.remove_edge(u, v)
        
# Simplify Graph: Remove nodes
G.remove_nodes_from(list(nx.isolates(G)))

G = ox.project_graph(G, to_crs='epsg:5070')
G = remove_uncenessary_nodes(G)

ox.plot_graph(G)
```


In [ ]:
def remove_uncenessary_nodes(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg == 0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg == 0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component) < 10:
            for node in component:
                _nodes_removed += 1
                network.remove_node(node)

    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed / float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))

    return network


# Obtain OSM Network in Harris County, Texas
G = ox.graph_from_place('Harris County, TX, USA', network_type='drive', simplify=True)

# Simplify Graph: Remove edges
for u, v, data in G.copy().edges(data=True):
    if data['highway'] not in ['motorway', 'motorway_link', 
                               'trunk', 'trunk_link',
                               'primary', 'primary_link', 
# This line is commented to make the future process faster. 
# If your study area is small and doesn't have enough road network, please uncomment the following line. 
#                                'secondary', 'secondary_link'  
                              ]:
        G.remove_edge(u, v)
        
# Simplify Graph: Remove nodes
G.remove_nodes_from(list(nx.isolates(G)))

G = ox.project_graph(G, to_crs='epsg:5070')
G = remove_uncenessary_nodes(G)

ox.plot_graph(G)

## Replicate Spatial accessiblity study

### 4. (1 ponit) Find the nearest OSM node from given locations
**4.1.** Examine the following function and make an appropriate change if necessary. <br>
**4.2.** Calculate the nearest osm node of `supply` and `demand` GeoDataFrame by calling `find_nearest_osm` function. 

```python
    
def find_nearest_osm(network, gdf):
    """
    # This function helps you to find the nearest OSM node from a given GeoDataFrame
    # If geom type is point, it will take it without modification, but 
    # IF geom type is polygon or multipolygon, it will take its centroid to calculate the nearest element. 
    
    Input: 
    - network (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - gdf (GeoDataFrame): stores locations in its `geometry` column 
    
    Output:
    - gdf (GeoDataFrame): will have `nearest_osm` column, which describes the nearest OSM node 
                          that was computed based on its geometry column
      
    """
    for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
        if row.geometry.geom_type == 'Point':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                                    X=row.geometry.x, 
                                                    Y=row.geometry.y
                                                   )
        elif row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                        X=row.geometry.centroid.x, 
                                        Y=row.geometry.centroid.y
                                       )
        else:
            print(row.geometry.geom_type)
            continue

        gdf.at[idx, 'nearest_osm'] = nearest_osm

    return gdf
```

In [ ]:
def find_nearest_osm(network, gdf):
    """
    # This function helps you to find the nearest OSM node from a given GeoDataFrame
    # If geom type is point, it will take it without modification, but 
    # IF geom type is polygon or multipolygon, it will take its centroid to calculate the nearest element. 
    
    Input: 
    - network (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - gdf (GeoDataFrame): stores locations in its `geometry` column 
    
    Output:
    - gdf (GeoDataFrame): will have `nearest_osm` column, which describes the nearest OSM node 
                          that was computed based on its geometry column
      
    """
    for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
        if row.geometry.geom_type == 'Point':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                                    X=row.geometry.x, 
                                                    Y=row.geometry.y
                                                   )
        elif row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                        X=row.geometry.centroid.x, 
                                        Y=row.geometry.centroid.y
                                       )
        else:
            print(row.geometry.geom_type)
            continue

        gdf.at[idx, 'nearest_osm'] = nearest_osm

    return gdf

# Your code here



In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert type(supply) == gpd.GeoDataFrame
assert type(demand) == gpd.GeoDataFrame
assert 'nearest_osm' in supply.columns
assert 'nearest_osm' in demand.columns

print('Success!')

### 5. (1 point) Measure spatial accessibility to hospitals by reusing the functions. 

**5.1.** Examine the following function and make an appropriate change if necessary. <br>
**5.2.** Measure spatial accessibility to hospitals in your own study area by calling `step1_2SFCA` and `step2_2SFCA` function, sequentially. <br>
**5.3.** Find an appropriate threshold travel distance (`threshold`) within your study area. This value can be determined by examining the accessibility measures. They should not have more than 25% of 0 value but create the distribution of accessibility measures more than 5 unique values. <br>

```python
def extract_edges_nodes_from_networkx(network):
    nodes, edges = ox.graph_to_gdfs(network, nodes=True, edges=True, node_geometry=True)
    
    return nodes, edges


def step1_2SFCA(supply, supply_attr, demand, demand_attr, mobility, threshold):
    """
    Input: 
    - supply (GeoDataFrame): stores locations and attributes of supply 
    - supply_attr (str): the column of `supply` to be used for the analysis
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - demand_attr (str): the column of `demand` to be used for the analysis
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - supply_ (GeoDataFrame): 
      a copy of supply and it stores supply-to-demand ratio of each supply at `ratio` column
    """
    
    # Extract the nodes and edges of the network dataset for the future analysis. 
    nodes, edges = extract_edges_nodes_from_networkx(mobility)
    
    supply_ = supply.copy(deep=True)
    supply_['ratio'] = 0
    
    for i in trange(supply.shape[0]):
    
        # Create a catchment area from a given location
        temp_nodes = nx.single_source_dijkstra_path_length(mobility, supply.loc[i, 'nearest_osm'], threshold, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:5070")

        # Calculate the population within the catchment area
        temp_demand = demand.loc[demand['geometry'].centroid.within(access_nodes[0]), demand_attr].sum()

        # Calculate the number of hospital beds in each hospital
        temp_supply = supply.loc[i, supply_attr]

        # Calculate the number of hospital beds available for 100,000 people 
        supply_.at[i, 'ratio'] = temp_supply / temp_demand * 100000
        
    supply_['ratio'].replace(np.inf, 0, inplace=True)
    
    return supply_


def step2_2SFCA(result_step1, demand, mobility, threshold):
    """
    Input: 
    - result_step1 (GeoDataFrame): stores locations and 'ratio' attribute that resulted in step1 
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - demand_ (GeoDataFrame): 
      a copy of demand and it stores the final accessibility measures of each demand location at `ratio` column
    """
    
    # Extract the nodes and edges of the network dataset for the future analysis. 
    nodes, edges = extract_edges_nodes_from_networkx(mobility)
    
    demand_ = demand.copy(deep=True)
    demand_['access'] = 0

    for j in trange(demand.shape[0]):
        temp_nodes = nx.single_source_dijkstra_path_length(mobility, demand.loc[j, 'nearest_osm'], threshold, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:5070")

        accum_ratio = result_step1.loc[result_step1['geometry'].within(access_nodes[0]), 'ratio'].sum()
        demand_.at[j, 'access'] = accum_ratio

    return demand_

```

In [ ]:
def extract_edges_nodes_from_networkx(network):
    nodes, edges = ox.graph_to_gdfs(network, nodes=True, edges=True, node_geometry=True)
    
    return nodes, edges


def step1_2SFCA(supply, supply_attr, demand, demand_attr, mobility, threshold):
    """
    Input: 
    - supply (GeoDataFrame): stores locations and attributes of supply 
    - supply_attr (str): the column of `supply` to be used for the analysis
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - demand_attr (str): the column of `demand` to be used for the analysis
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - supply_ (GeoDataFrame): 
      a copy of supply and it stores supply-to-demand ratio of each supply at `ratio` column
    """
    
    # Extract the nodes and edges of the network dataset for the future analysis. 
    nodes, edges = extract_edges_nodes_from_networkx(mobility)
    
    supply_ = supply.copy(deep=True)
    supply_['ratio'] = 0
    
    for i in trange(supply.shape[0]):
    
        # Create a catchment area from a given location
        temp_nodes = nx.single_source_dijkstra_path_length(mobility, supply.loc[i, 'nearest_osm'], threshold, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:5070")

        # Calculate the population within the catchment area
        temp_demand = demand.loc[demand['geometry'].centroid.within(access_nodes[0]), demand_attr].sum()

        # Calculate the number of hospital beds in each hospital
        temp_supply = supply.loc[i, supply_attr]

        # Calculate the number of hospital beds available for 100,000 people 
        supply_.at[i, 'ratio'] = temp_supply / temp_demand * 100000
        
    supply_['ratio'].replace(np.inf, 0, inplace=True)
    
    return supply_


def step2_2SFCA(result_step1, demand, mobility, threshold):
    """
    Input: 
    - result_step1 (GeoDataFrame): stores locations and 'ratio' attribute that resulted in step1 
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - demand_ (GeoDataFrame): 
      a copy of demand and it stores the final accessibility measures of each demand location at `ratio` column
    """
    
    # Extract the nodes and edges of the network dataset for the future analysis. 
    nodes, edges = extract_edges_nodes_from_networkx(mobility)
    
    demand_ = demand.copy(deep=True)
    demand_['access'] = 0

    for j in trange(demand.shape[0]):
        temp_nodes = nx.single_source_dijkstra_path_length(mobility, demand.loc[j, 'nearest_osm'], threshold, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:5070")

        accum_ratio = result_step1.loc[result_step1['geometry'].within(access_nodes[0]), 'ratio'].sum()
        demand_.at[j, 'access'] = accum_ratio

    return demand_


In [ ]:
# Your code here
dist = 

step1 = step1_2SFCA()
step2 = step2_2SFCA()


In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert len(step2['access'].unique()) > 5
assert step2.loc[step2['access'] == 0].shape[0] / step2.shape[0] < 0.25

fig, ax = plt.subplots(figsize=(10,10))

step1.plot(markersize='BEDS', ax=ax, zorder=2, color='black')
step2.plot('access', ax=ax, legend=True, cmap='Blues', scheme='FisherJenks', zorder=1)
step2.loc[step2['access'] == 0].plot(ax=ax, color='grey', zorder=1)
step2.boundary.plot(ax=ax, linestyle='dotted', lw=0.5, color='black', zorder=2)

plt.show()

print('Success!')